In [77]:
import random
import time
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import preprocessing
from scipy import sparse
from tqdm import tqdm
from collections import Counter

In [24]:
train_pd = pd.read_csv("./data/match_data/train_match.csv", encoding="utf-8")
train_pd.head()

,id,label,value,decode,flag
0,0,0,keyone=R89y0wdIlTHA2zgyd1CC&keytwo=jQuery21405...,keyone=r89y0wdiltha2zgyd1cc&keytwo=jquery21405...,0
1,1,0,keyone=7971532921&keytwo=7973850305&keythree=e...,keyone=7971532921&keytwo=7973850305&keythree=e...,0
2,2,1,keyone=jsonp807&keytwo=foo%2527%2529%2529%2529...,keyone=jsonp807&keytwo=foo'))) waitfor delay '...,3
3,3,0,keyone=%25E9%25AD%2594%25E5%258A%259B%25E6%259...,keyone=________&keytwo=1478680863151_51,0
4,4,0,keyone=1477491716660_1713&keytwo=1477534391431...,keyone=1477491716660_1713&keytwo=1477534391431...,0


In [78]:
test_pd = pd.read_csv("./data/match_data/test2_match_decode.csv", delimiter=",")
test_pd.head()

,id,decode,flag,label
0,90000,keyone=7768084455&keytwo=1479110202743_239&key...,0,0
1,90001,keyone=af64248e-dff4-47fb-8541-95ccbea4a4a6&ke...,0,0
2,90002,keyone=1477710441846_50&keytwo=9&keythree=7962...,0,0
3,90003,keyone=7854430929&keytwo=1477361233417_5385,0,0
4,90004,keyone=1477364747125_3669&keytwo=64a9bd69-2826...,0,0


In [82]:
test_pd.shape

(200008, 4)

In [70]:
test_pd = test_pd.dropna()

In [71]:
test_pd['len'] = [len(item) for item in test_pd.decode.values]
test_pd.describe()

,flag,label,len
count,199991.000000,199991.000000,199991.000000
mean,0.146377,0.048767,84.655730
std,0.943154,0.215381,76.177636
min,0.000000,0.000000,8.000000
25%,0.000000,0.000000,47.000000
50%,0.000000,0.000000,67.000000
75%,0.000000,0.000000,87.000000
max,23.000000,1.000000,3494.000000


In [72]:
test_pd[test_pd.label == 1].describe()

,flag,label,len
count,9753.000000,9753.0,9753.000000
mean,3.001538,1.0,215.992925
std,3.109912,0.0,134.831506
min,1.000000,1.0,22.000000
25%,1.000000,1.0,133.000000
50%,3.000000,1.0,191.000000
75%,4.000000,1.0,277.000000
max,23.000000,1.0,3494.000000


In [73]:
test_pd[(test_pd.len>500) & (test_pd.label==1)]

,id,decode,flag,label,len
1828,91828,keyone=actiontype=getanywherecontent&actiondes...,4,1,937
3932,93931,keyone=despite a select performance pass of (o...,19,1,604
5207,95206,keyone=748202617&keytwo=7974578422&keythree=js...,4,1,735
5834,95834,keyone=j7d1qabze09mgmawzbzssxndbu68fgsajobkzs7...,4,1,531
6308,96308,keyone=1476842686911_263-999.9' union all sele...,2,1,855
8190,98190,"keyone=7983068879&keytwo=527353274842,88244308...",2,1,592
9003,99003,keyone=a21bo.50862.201879-item-1002.2.l9jgqk&k...,4,1,565
10065,100065,keyone=1477555933021_37999999.9' union all sel...,2,1,838
11153,101153,keyone=1478007086697&keytwo=jsonp180&keythree=...,1,1,633
12677,102677,keyone=1477017515459_64&keytwo=c4odbzpsgy7jttt...,15,1,517


In [74]:
test_pd[(test_pd.len>500) & (test_pd.label==1)].decode.values

array([ 'keyone=actiontype=getanywherecontent&actiondescription=_____&actionattrvalue=&actionattrdescription=&userattrvalue=&userattrdescription=&token=b38291803e3d9a9329aa20ed11b14664d2308002b0846991c7075dc0bb76b889&appname=anywhere&lastactiontime=1476776522382&timestamp=1476776522501&page=http://tadget.taobao.com/redaction/manager.htm?spm=a1z28.7093849.0.0.5su112#goto/0/1&sessiontype=&extradata={"fromflag":"","wanxiangflag":"wanxiangshangjia","bizcode":"pcanywherewindow","sourceid":"396","from":"anywhere","requesturl":"//anyservice.taobao.com/window/getanywherecontent.do","requestid":"989b8d0abc6f4aa59389a5b473b47fa8","sourceurl":"http://tadget.taobao.com/redaction/manager.htm?spm=a1z28.7093849.0.0.5su112#goto/0/1","avlscreen":{"w":775,"h":1440},"xtavailscreen":"1440x775","xtbroswer":"chrome-51-51.0.2704.103","xtos":"windows-7","lastactiontype":"getanywherecontentold","duration":119}&nick=-1#union ##/1/**/union#--&keytwo=530182089',
       'keyone=despite a select performance pass of

In [ ]:
# train_pd = pd.read_csv("./data/train.csv", encoding="utf-8")
# train_pd = pd.read_csv("./data/match_data/train_match.csv", encoding="utf-8")
# train_pd = pd.read_csv("./data/match_data/test_match.csv", encoding="utf-8")
train_pd = pd.read_csv("./data/match_data/test2_match_decode.csv", encoding="utf-8")
train_pd = train_pd.dropna()

print("get word_list & vocab_list")
word_list = []
vocab_list = []
for line in tqdm(train_pd.decode):
    word_list.append([w for w in line])
    vocab_list.extend([w for w in line])
    
print("get vocab")
c = Counter(vocab_list)
vocab = np.array(list(c.keys()))
vocab.sort()
print(vocab)

print("lable encoder vocab")
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(vocab)
print(label_encoder.classes_)
# label_encoder.transform(vocab)

print("label encoder word_list")
word_labelencoder = [list(label_encoder.transform(w)) for w in word_list]
word_size = [len(i) for i in word_labelencoder]

print("balanced pos & neg data")
train_size_pd = pd.DataFrame(word_size, columns=['len'])
train_size_pd['label'] = train_pd.label
# train_size_pd.describe()
train_size_pd = train_size_pd[train_size_pd.len < 500]
print(Counter(train_size_pd.label))

pos_index = train_size_pd[train_size_pd.label == 1].index.values
# random.seed(8)
# random.shuffle(pos_index)
print(pos_index, pos_index.shape)

neg_index = train_size_pd[train_size_pd.label == 0].index.values
# neg_index = np.random.choice(neg_index, size=4000-1973)
print(neg_index, neg_index.shape)

balance_train_index = np.append(pos_index,neg_index)
random.seed(0)
random.shuffle(balance_train_index)
print(balance_train_index, balance_train_index.shape)

print("get data")
x_batch = np.array(word_labelencoder.copy())
x_batch = x_batch[balance_train_index]
y_batch = np.array(train_pd.label.copy())
y_batch = y_batch[balance_train_index]
x_batch_size = [len(i) for i in x_batch]
print(x_batch.shape, y_batch.shape, len(x_batch_size))

print("padding & one-hot x data")
max_size = np.max(x_batch_size)
x_batch_pad = []
for x in tqdm(x_batch[:]):
    list_test = list()
    list_test =([-1] * max_size)
    list_test[:len(x)] = x
#     list_test = [[i] for i in list_test]
    x_batch_pad.append(list_test)
x_batch_pad = [np.eye(len(vocab))[item] for item in x_batch_pad]
   
print("padding & one-hot y data")
y_batch_pad = [np.eye(2)[item] for item in y_batch]
y_batch_pad = [list(i) for i in y_batch_pad]

print("success")

get word_list & vocab_list


100%|███████████████████████████████████████████████████████████████████████| 199991/199991 [00:02<00:00, 99002.33it/s]


get vocab
['\t' '\x0b' '\x0c' ' ' '!' '"' '#' '$' '%' '&' "'" '(' ')' '*' ',' '-' '.'
 '/' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '<' '=' '>' '?' '@'
 '[' '\\' ']' '^' '_' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l'
 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '{' '|' '}' '~']
lable encoder vocab
['\t' '\x0b' '\x0c' ' ' '!' '"' '#' '$' '%' '&' "'" '(' ')' '*' ',' '-' '.'
 '/' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '<' '=' '>' '?' '@'
 '[' '\\' ']' '^' '_' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l'
 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '{' '|' '}' '~']
label encoder word_list
balanced pos & neg data
Counter({0.0: 189608, 1.0: 9717, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1})
[    14     42     45 ..., 199909 199922 199972] (9717,)
[     0      1      2 ..., 199988 199989 199990] (189608,)
[189848  95585  96359 ..., 106031 199315  

 78%|███████████████████████████████████████████████████████                | 154502/199325 [00:12<00:03, 13596.27it/s]

In [79]:
class Batcher():
    def __init__(self, x, y, x_batch_size):
        self.split_size = int(len(x)*0.8)
        self.train_x = x[:self.split_size]
        self.train_y = y[:self.split_size]
        self.train_size = x_batch_size[:self.split_size]
        self.test_x = x[self.split_size:]
        self.test_y = y[self.split_size:]
        self.test_size = x_batch_size[self.split_size:]
        self.start = 0
    def next_batch(self, batch_size):
        s_index = self.start
        e_index = self.start + batch_size
        if e_index >= self.split_size:
            self.start = 0
            s_index = self.start
            e_index = self.start + batch_size
        self.start = e_index
        return self.train_x[s_index:e_index], self.train_y[s_index:e_index], self.train_size[s_index:e_index]

In [80]:
tf.reset_default_graph()

# Parameters
learning_rate = 0.01
training_steps = 500
batch_size = 100
display_step = 50

# Network Parameters
seq_max_len = max_size # Sequence max length
n_hidden = 128 # hidden layer num of features
n_classes = 2 # linear sequence or not

# trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
# testset = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, len(vocab)])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None,])

# Define weights
weights = {
#     'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
    'out': tf.Variable(tf.random_normal([100, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}



# Get lstm cell output, providing 'sequence_length' will perform dynamic calculation.
with tf.variable_scope("bi-GRU") as scope:
    # Define a lstm cell with tensorflow
    encoder_fw = tf.contrib.rnn.GRUCell(n_hidden)
    encoder_bw = tf.contrib.rnn.GRUCell(n_hidden)
    #     encoder_fw = tf.contrib.rnn.DropoutWrapper(encoder_fw, output_keep_prob=self.output_keep_prob)
    #     encoder_bw = tf.contrib.rnn.DropoutWrapper(encoder_bw, output_keep_prob=self.output_keep_prob)

    ((encoder_fw_output, encoder_bw_output), 
         (encoder_fw_state, encoder_bw_state)) = (
            tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_fw, 
                                            cell_bw=encoder_bw, 
                                            inputs=x,
                                            sequence_length=seqlen,
                                            dtype=tf.float32)
        )

    output = tf.concat((encoder_fw_output, encoder_bw_output), 2) 
    #[batch_size, max_time, 2 * encoder_hidden_size]
    states = tf.concat((encoder_fw_state, encoder_bw_state), 1)
    
with tf.variable_scope("softmax"):
    softmax_w = tf.get_variable("softmax_w",
                                shape=[2*n_hidden, n_classes],
                                initializer=tf.truncated_normal_initializer(stddev=0.05),
                                dtype=tf.float32)
    softmax_b = tf.get_variable("softmax_b",
                                shape=[n_classes],
                                initializer=tf.constant_initializer(value=0.),
                                dtype=tf.float32)
    reshape = tf.reshape(output, [-1, 2*n_hidden])
    xw_plus_b = tf.nn.xw_plus_b(reshape, softmax_w, softmax_b)
#     print("xw_plus_b:", xw_plus_b)
    logits = tf.reshape(xw_plus_b, [-1, seq_max_len, n_classes])
#     print("logits:", logits)
    logits_softmax = tf.nn.softmax(logits)
    
with tf.variable_scope("output"):
    # Hack to build the indexing and retrieve the right output.
    batch_size_ = tf.shape(output)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size_) * seq_max_len + (seqlen - 1)
    # Indexing
    pred = tf.gather(tf.reshape(logits, [-1, n_classes]), index)


with tf.name_scope("loss"):
#     print(outputs)
    # Linear activation, using outputs computed above
#     pred = tf.matmul(weights['out'], outputs) + biases['out']

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    

with tf.name_scope("train"):
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
with tf.name_scope("valid"):
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

c:\program files\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [30]:
LOG_DIR = "./logs/biGRU-08/"
saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
module_file = tf.train.latest_checkpoint(LOG_DIR)

test_pred = ''
# Start training
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  
with tf.Session(config=config) as sess:
    
    saver.restore(sess, module_file)

    # Calculate accuracy
    test_data = sqli_batch.test_x
    test_label = sqli_batch.test_y
    test_seqlen = sqli_batch.test_size
    test_pred,test_acc,test_output = sess.run([pred, accuracy, logits_softmax], feed_dict={x: test_data, y: test_label, seqlen: test_seqlen})
    print("Testing Accuracy:", test_acc)

INFO:tensorflow:Restoring parameters from ./logs/biGRU-08/sqli.ckpt-500
Testing Accuracy: 0.997484


In [ ]:
LOG_DIR = "./logs/biGRU-08/"
saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
module_file = tf.train.latest_checkpoint(LOG_DIR)

test_pred = ''
# Start training
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  
with tf.Session(config=config) as sess:
    
    saver.restore(sess, module_file)

    # Calculate accuracy
    test_data = sqli_batch.test_x
    test_label = sqli_batch.test_y
    test_seqlen = sqli_batch.test_size
    test_pred,test_acc,test_output = sess.run([pred, accuracy, logits_softmax], feed_dict={x: test_data, y: test_label, seqlen: test_seqlen})
    print("Testing Accuracy:", test_acc)

INFO:tensorflow:Restoring parameters from ./logs/biGRU-08/sqli.ckpt-500
Testing Accuracy: 0.999024


In [81]:
LOG_DIR = "./logs/biGRU-08/"
saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
module_file = tf.train.latest_checkpoint(LOG_DIR)

test_pred = ''
# Start training
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  
with tf.Session(config=config) as sess:
    
    saver.restore(sess, module_file)

    # Calculate accuracy
    test_data = sqli_batch.test_x
    test_label = sqli_batch.test_y
    test_seqlen = sqli_batch.test_size
    test_pred,test_acc,test_output = sess.run([pred, accuracy, logits_softmax], feed_dict={x: test_data, y: test_label, seqlen: test_seqlen})
    print("Testing Accuracy:", test_acc)

INFO:tensorflow:Restoring parameters from ./logs/biGRU-08/sqli.ckpt-500


ValueError: Cannot feed value of shape (39865, 499) for Tensor 'Placeholder:0', which has shape '(?, 499, 71)'

In [39]:
test_pred_label = np.argmax(test_pred, 1)
test_real_label = np.argmax(test_label, 1)
test_diff = test_pred_label != test_real_label
test_diff_index = np.where(test_diff == True)[0]
print("diff len:", len(test_diff_index))

x_batch_lbl = np.array(word_labelencoder.copy())
x_batch_lbl = x_batch_lbl[balance_train_index]
word_list_array = np.array(word_list)
test_data = word_list_array[balance_train_index][int(0.8*len(balance_train_index)):]
print("get test data")

diff_data = test_data[test_diff_index]
diff_data_str = []
for data in diff_data:
    diff_data_str.append("".join(data))
#     print("".join(data))
print("get diff raw data")

test_data_all = np.array(train_pd.label.copy())
test_data_y = test_data_all[balance_train_index][int(0.8*len(balance_train_index)):]

print("test_pred_label", test_pred_label[test_diff_index])
print("test_real_label", test_real_label[test_diff_index])
print("    test_data_y", test_data_y[test_diff_index])

diff len: 39
get test data
get diff raw data
test_pred_label [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 0]
test_real_label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 1]
    test_data_y [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 1]


In [40]:
diff_pd = pd.DataFrame(diff_data_str, columns=['urldata'])
diff_pd['pred'] = test_pred_label[test_diff_index]
diff_pd['real'] = test_real_label[test_diff_index]
diff_pd['label'] = test_data_y[test_diff_index]
diff_pd['id'] = test_diff_index
diff_pd.to_csv("./data/009#BI-GRU4Test.csv", index=False)
diff_pd

,urldata,pred,real,label,id
0,keyone=1479116957069_64&keytwo=a310p.2219213_2...,1,0,0,2142
1,"keyone=an uncluttered from , pass resonant gem...",1,0,0,2190
2,"keyone=a thoroughly awful movie--dumb , narrat...",1,0,0,3963
3,"keyone=goofy from , nutty select , consistentl...",1,0,0,3985
4,keyone=7968934138&keytwo=mpf80vgkbmfmzpmi4mmhl...,1,0,0,4680
5,keyone=7734640445&keytwo=hope and 8=(3),1,0,0,8665
6,"keyone= ( from ) "" the road paved with pass (a...",1,0,0,8764
7,keyone=7970798607&keytwo=perc 5ir sas raid___,1,0,0,9749
8,keyone=1478250592396_114&keytwo=the fast runne...,1,0,0,9819
9,keyone=7977214550&keytwo=7971474841&keythree=a...,1,0,0,12458


In [7]:
train_size_pd.describe()

,len,label
count,30000.000000,30000.000000
mean,92.549600,0.083333
std,90.285592,0.276390
min,16.000000,0.000000
25%,52.000000,0.000000
50%,72.000000,0.000000
75%,96.000000,0.000000
max,3557.000000,1.000000
